카카오로컬api활용

In [1]:
import requests
from urllib.parse import quote #한글로 보낼 때 오류 제거
import pandas as pd


api key 가져오기

In [2]:
from google.colab import files
up = files.upload()

Saving kakaoapikey.txt to kakaoapikey.txt


In [3]:
with open('kakaoapikey.txt') as kf:
    kakao_key = kf.read()


url 만들기

In [4]:
local_url = "https://dapi.kakao.com/v2/local/search/address.json"
addr = '광주광역시 서구 내방로 111(치평동)'
url = f'{local_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EA%B4%91%EC%A3%BC%EA%B4%91%EC%97%AD%EC%8B%9C%20%EC%84%9C%EA%B5%AC%20%EB%82%B4%EB%B0%A9%EB%A1%9C%20111%28%EC%B9%98%ED%8F%89%EB%8F%99%29'

api key를 입력하기 위한 헤더를 만들어 줌

In [5]:
header = { 'Authorization': f'KakaoAK {kakao_key}' }

In [6]:
result = requests.get( url, headers = header) 
result , type(result)

(<Response [200]>, requests.models.Response)

In [7]:
result = requests.get( url, headers = header).json()
result

{'documents': [{'address': {'address_name': '광주 서구 치평동 1200',
    'b_code': '2914012000',
    'h_code': '2914074500',
    'main_address_no': '1200',
    'mountain_yn': 'N',
    'region_1depth_name': '광주',
    'region_2depth_name': '서구',
    'region_3depth_h_name': '치평동',
    'region_3depth_name': '치평동',
    'sub_address_no': '',
    'x': '126.851461925213',
    'y': '35.1600994105234'},
   'address_name': '광주 서구 내방로 111',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '광주 서구 내방로 111',
    'building_name': '광주광역시청',
    'main_building_no': '111',
    'region_1depth_name': '광주',
    'region_2depth_name': '서구',
    'region_3depth_name': '치평동',
    'road_name': '내방로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.851461925213',
    'y': '35.1600994105234',
    'zone_no': '61945'},
   'x': '126.851461925213',
   'y': '35.1600994105234'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [8]:
result.keys()

dict_keys(['documents', 'meta'])

In [9]:
result['documents'][0].keys()

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

In [10]:
#경도
lng = float(result['documents'][0]['x'])
lng

126.851461925213

In [11]:
#위도
lat = float(result['documents'][0]['y'])
lat

35.1600994105234

광주관공서좌표채우기

In [12]:
up = files.upload()

Saving 광주광역시내_주요기관.csv to 광주광역시내_주요기관.csv


In [13]:
df=pd.read_csv('광주광역시내_주요기관.csv')
df

,장소,도로명주소
0,광주광역시청,광주광역시 서구 내방로 111(치평동)
1,광주광역시 동구청,광주광역시 동구 서남로 1(서석동)
2,광주광역시 북구청,광주광역시 북구 우치로 77(용봉동)
3,광주광역시 서구청,광주광역시 서구 경열로 33(농성동)
4,광주광역시 남구청,광주광역시 남구 봉선로 1(봉선동)
5,광산구청,광주광역시 광산구 광산로29번길 15(송정동)


In [14]:
lng_list, lat_list = [],[]
for i in df.index:
    addr = df['도로명주소'][i]
    url = f'{local_url}?query={quote(addr)}'
    result = requests.get( url, headers = header).json()

    lng = float(result['documents'][0]['x'])
    lat = float(result['documents'][0]['y'])
    lng_list.append(lng)
    lat_list.append(lat)


In [15]:
lng_list

[126.851461925213,
 126.92309316107,
 126.912107567985,
 126.890112188537,
 126.902447908848,
 126.79364427408]

In [16]:
# df.insert(2,'경도(lng)',lng_list)
df['경도']=lng_list

In [17]:
# df.insert(3,'위도(lat)',lat_list)
df['위도']=lat_list

In [18]:
df

,장소,도로명주소,경도(lng),위도(lat)
0,광주광역시청,광주광역시 서구 내방로 111(치평동),126.851462,35.160099
1,광주광역시 동구청,광주광역시 동구 서남로 1(서석동),126.923093,35.146142
2,광주광역시 북구청,광주광역시 북구 우치로 77(용봉동),126.912108,35.174291
3,광주광역시 서구청,광주광역시 서구 경열로 33(농성동),126.890112,35.152029
4,광주광역시 남구청,광주광역시 남구 봉선로 1(봉선동),126.902448,35.132906
5,광산구청,광주광역시 광산구 광산로29번길 15(송정동),126.793644,35.139647
